In [2]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '5'
n_gpus = len(os.environ['CUDA_VISIBLE_DEVICES'].split(','))
model_name = 'model_nasnet'

import keras.backend as K
import tensorflow as tf

# gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.7)
# sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options))

# 导入必要的库

In [3]:
import numpy as np
import pandas as pd
from keras.layers import *
from keras.models import *
from keras.optimizers import *
from keras.applications import *
from keras.regularizers import l2

from keras.preprocessing.image import *

import matplotlib.pyplot as plt
import random
import os
import cv2
from tqdm import tqdm
from glob import glob
import multiprocessing

from sklearn.model_selection import train_test_split
from collections import Counter
from keras import backend as K
from keras.utils import multi_gpu_model

from IPython.display import display, Image

%matplotlib inline

%config InlineBackend.figure_format = 'retina'

In [4]:
df = pd.read_csv('sample_ready/Annotations/label.csv', header=None)
df.columns = ['filename', 'label_name', 'label']
df = df.sample(frac=1).reset_index(drop=True) # shuffle

df.label_name = df.label_name.str.replace('_labels', '')

c = Counter(df.label_name)

label_count = dict([(x, len(df[df.label_name == x].label.values[0])) for x in c.keys()])
label_names = list(label_count.keys())

In [5]:
print(label_names)

['upper_body', 'lower_body']


In [6]:
print(label_count)

{'upper_body': 7, 'lower_body': 2}


In [8]:
width = 331

def load_model():
    base_model = NASNetLarge(weights='imagenet', input_shape=(width, width, 3), include_top=False, pooling='avg')

    input_tensor = Input((width, width, 3))
    x = input_tensor
    x = Lambda(nasnet.preprocess_input)(x)
    x = base_model(x)
    x = Dropout(0.5)(x)
    x = [Dense(count, activation='softmax', name=name)(x) for name, count in label_count.items()]

    model = Model(input_tensor, x)
    return model

model = load_model()
model.load_weights('model_nasnet.h5')
# model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])

# from IPython.display import SVG
# from keras.utils.vis_utils import model_to_dot, plot_model

# plot_model(model, show_shapes=True, to_file='model_simple.png')
# SVG(model_to_dot(model, show_shapes=True).create(prog='dot', format='svg'))

# 在测试集上预测

In [22]:
df_test = pd.read_csv('rank/Tests/question.csv', header=None)
df_test.columns = ['filename', 'label_name', 'label']

fnames_test = df_test.filename

n_test = len(df_test)
df_test.head()

,filename,label_name,label
0,Tests/Images/polo.png,upper_body_labels,ynnnnnn
1,Tests/Images/polo衬衫短袖.png,upper_body_labels,ynnnnnn
2,Tests/Images/彩色卫衣.png,upper_body_labels,nnnnnyn
3,Tests/Images/短裤全身.png,lower_body_labels,ny
4,Tests/Images/格子衬衣.png,upper_body_labels,nynnnnn


In [23]:
def f(index):
    return index, cv2.resize(cv2.imread('rank/'+fnames_test[index]), (width, width))

X_test = np.zeros((n_test, width, width, 3), dtype=np.uint8)
with multiprocessing.Pool(12) as pool:
    with tqdm(pool.imap_unordered(f, range(n_test)), total=n_test) as pbar:
        for i, img in pbar:
            X_test[i] = img[:,:,::-1]

100%|██████████| 11/11 [00:00<00:00, 137.48it/s]


In [24]:
y_pred = model.predict(X_test, batch_size=128, verbose=1)

11/11 [==============================] - 1s 47ms/step


In [25]:
for i in range(n_test):
    problem_name = df_test.label_name[i].replace('_labels', '')
    problem_index = label_names.index(problem_name)
    probs = y_pred[problem_index][i]
    df_test.label[i] = ';'.join(np.char.mod('%.8f', probs))

In [26]:
print(df_test)

                     filename         label_name  \
0       Tests/Images/polo.png  upper_body_labels   
1   Tests/Images/polo衬衫短袖.png  upper_body_labels   
2       Tests/Images/彩色卫衣.png  upper_body_labels   
3       Tests/Images/短裤全身.png  lower_body_labels   
4       Tests/Images/格子衬衣.png  upper_body_labels   
5         Tests/Images/卫衣.png  upper_body_labels   
6       Tests/Images/西装全身.jpg  lower_body_labels   
7       Tests/Images/西装全身.jpg  upper_body_labels   
8       Tests/Images/西装t恤.png  upper_body_labels   
9        Tests/Images/西装裤.png  lower_body_labels   
10      Tests/Images/印花短袖.png  upper_body_labels   

                                                label  
0   0.99166542;0.00083193;0.00000981;0.00013476;0....  
1   0.00001133;0.99915481;0.00001920;0.00000296;0....  
2   0.00006084;0.77797103;0.00047777;0.01878709;0....  
3                               0.00085786;0.99914217  
4   0.00008899;0.99899071;0.00046693;0.00002680;0....  
5   0.00059175;0.00189139;0.00035299;0.

In [27]:
fname_csv = 'pred_test.csv'
df_test.to_csv(fname_csv, index=None, header=None)